In [28]:
# Warning Management
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# DataFrames
import pandas as pd

# Plotting
import matplotlib.pyplot as plt

# Maths
import numpy as np
import math

# Path management
import pathlib
import os
import sys

# Progress tracking
from tqdm import tqdm

# Regular Expressions
import re

# Logging
import logging

# Helpers
from helper_fns import convert_vin_valid, return_matched_vins, create_valid_zip, try_divide

# 1. Import Data

In [8]:
# Version of raw data to get
version = "110723"

# Paths 
path = pathlib.Path().resolve().parent.parent/ "rn_home"

# Set up paths for different systems
if sys.platform == 'linux':
    data_path = path / "data" 
    matched_municipal_path = data_path / "municipal_dataset_matched" / f"municipal_dataset_matched_{version}.csv"
    rlpolk_path = data_path / "rlpolk_data" / "rlpolk_data_matched.csv"
else:
    data_path = path.parent / "data"
    print("YET TO FIX THIS")

In [ ]:
matched_municipal_data = pd.read_csv(matched_municipal_path, index_col = [0])
matched_rlpolk_data = pd.read_csv(rlpolk_path, index_col = 0)
matched_rlpolk_data["year"] = matched_rlpolk_data["YEAR_MONTH"].astype(str).str[0:4].astype(int)
matched_rlpolk_data["month"] = matched_rlpolk_data["YEAR_MONTH"].astype(str).str[4:].astype(int)

# 2. Comparison

## Set up data

In [50]:
# Get 2021 data for the municipal dataset
matched_municipal_21 = matched_municipal_data[matched_municipal_data["record_from"].str.contains("_MV_21")]

# Drop duplicates
matched_municipal_21_dd = matched_municipal_21.drop_duplicates("vehicle_id")

# Print difference
print(f"There is a difference of {len(matched_municipal_21) - len(matched_municipal_21_dd)} when deduplicating by VINs")

There is a difference of 28152 when deduplicating by VINs


In [14]:
# Get 2021 data for RLPolk
matched_rlpolk_21_filter = ((matched_rlpolk_data["year"] == 2020) & (matched_rlpolk_data["month"] >=7)) | ((matched_rlpolk_data["year"] == 2021)  & (matched_rlpolk_data["month"] <7))
matched_rlpolk_21 = matched_rlpolk_data.loc[matched_rlpolk_21_filter]

## Comparisons

### Compare the number of vehicles between municipal and RLPolk

### Define filters

In [58]:
def get_lease_mask(data):
    mask = data["lease_street"].notna()| data["lease_city"].notna() | data["lease_state"].notna() | data["lease_zip"].notna()
    return mask

def get_model_year_mask(data):
    mask = (data["Model Year"] == 2020) | (data["Model Year"] ==2021) | (data["Model Year"] ==2019)
    return mask

def get_matched_mask(data):
    mask = (data["Manufacturer Name"].notna())
    return mask

def get_rlp_mask(data):
    rlpolk_models = matched_rlpolk_21["Model"].unique().tolist()
    mask = data["Model"].isin(rlpolk_models)
    return mask

In [39]:
# Filter for leased vehicles
lease_mask = get_lease_mask(matched_municipal_21)

# Get model years 20 and 2021
model_year_mask = get_model_year_mask(matched_municipal_21)

# Filter for matches
matched_mask = get_matched_mask(matched_municipal_21)

# Filter for CT
ct_mask = (matched_municipal_21["state"] == "CT") | (matched_municipal_21["zip_corrected"].astype(str).str[:-2].str.zfill(5).str[0:2] == "06")

In [43]:
# Print results
print(f"In the municipal dataset, {sum(~lease_mask & mun_veh_year_mask & matched_mask)} vehicles are sold in tax year 2021")
print(f"In the municipal dataset, {sum(mun_veh_year_mask & matched_mask)} vehicles are sold in tax year 2021, including leases")
print(f"This is calculated by taking tax year 2021, and filtering for model years > 2020 and 2021")

In the municipal dataset, 392137 vehicles are sold in tax year 2021
In the municipal dataset, 455306 vehicles are sold in tax year 2021, including leases
This is calculated by taking tax year 2021, and filtering for model years > 2020 and 2021


In [42]:
# Do the same for RLPolk
print(f"In the RLPolk Dataset, {matched_rlpolk_21["VEH_COUNT"].sum()} vehicles are sold in FY 2021")

# Try for the CY
print(f"In the RLPolk Dataset, {matched_rlpolk_data[matched_rlpolk_data["year"]==2021]["VEH_COUNT"].sum()} vehicles are sold in CY 2021")

In the RLPolk Dataset, 191187 vehicles are sold in FY 2021
In the RLPolk Dataset, 173449 vehicles are sold in CY 2021


### Compare only those models present in both datasets

In [59]:
# Filter for being in RLP 
mun_rlpolk_model_filter = get_rlp_mask(matched_municipal_21)
num_veh_rlpolk_mun = num_veh_mun & mun_rlpolk_model_filter
num_veh_rlpolk_mun_notlease = num_veh_rlpolk_mun & ~lease_mask

# Print results
print(f"In the municipal dataset, {num_veh_rlpolk_mun.sum()} vehicles **with the same models as RLPolk** sold in 2021")
print(f"In the municipal dataset, {num_veh_rlpolk_mun_notlease.sum()} vehicles **with the same models as RLPolk** sold in 2021, excl. leases")

In the municipal dataset, 425607 vehicles **with the same models as RLPolk** sold in 2021
In the municipal dataset, 365708 vehicles **with the same models as RLPolk** sold in 2021, excl. leases


### Compare EVs

In [56]:
num_veh_rlpolk_mun_evs = num_veh_mun & mun_rlpolk_model_filter & (matched_municipal_21["Fuel Type - Primary"] == "Electric")
num_veh_rlpolk_mun_evs_nl = num_veh_rlpolk_mun_evs & ~lease_mask

# Print results
print(f"In the municipal dataset, {num_veh_rlpolk_mun_evs.sum()} EVs **with the same models as RLPolk** sold in 2021")
print(f"In the municipal dataset, {num_veh_rlpolk_mun_evs_nl.sum()} EVs **with the same models as RLPolk** sold in 2021, excl. leases")

In the municipal dataset, 10505 EVs **with the same models as RLPolk** sold in 2021
In the municipal dataset, 8545 EVs **with the same models as RLPolk** sold in 2021, excl. leases


In [57]:
matched_rlpolk_21_evs = matched_rlpolk_21["Fuel Type - Primary"] == "Electric"

# Do the same for RLPolk
print(f"In the RLPolk Dataset, {matched_rlpolk_21[matched_rlpolk_21_evs]["VEH_COUNT"].sum()} EVs sold in FY 2021")

# Try with just calendar year
print(f"In the RLPolk Dataset, {matched_rlpolk_data[(matched_rlpolk_data["year"]==2021) & (matched_rlpolk_data["Fuel Type - Primary"]=="Electric")]["VEH_COUNT"].sum()} EVs sold in CY 2021")

In the RLPolk Dataset, 6086 EVs sold in FY 2021
In the RLPolk Dataset, 8461 EVs sold in CY 2021


### Compare EVs with CT state

In [115]:
len(matched_municipal_21[ct_mask & mun_rlpolk_model_filter &  (matched_municipal_21["Fuel Type - Primary"] == "Electric")])

12193